# Project Plan

- [ ] Acquire Avax Price Data
-   [ ] Acquire historical data from [CoinMarketCap](https://coinmarketcap.com/)
-   [ ] Acquire live data from [CoinMarketCap](https://coinmarketcap.com/) 
- [ ] Use Kats to build predictive forecast model


In [73]:

import json
import pandas as pd
import datetime
import plotly.express as px
import matplotlib.pyplot as plt
import matplotlib
import kats
#import warnings
from kats.models.sarima import SARIMAParams
import statsmodels.api as sm
#warnings.simplefilter(action='ignore')

In [74]:

today =  datetime.datetime.today().strftime('%Y-%m-%d')


In [75]:
avax_data = pd.read_csv('avalanche_2020-9-26_2022-2-26.csv').iloc[::-1]
avax_data.head()
avax_data['Average Daily Price'] = avax_data[['Open', 'Close']].mean(axis=1)
avax_daily_ave = avax_data[['Date','Average Daily Price']]
avax_daily_ave['Date'] = pd.to_datetime(avax_daily_ave['Date'])
avax_daily_ave.set_index('Date', inplace=True)
avax_daily_ave.head()
avax_daily_ave.tail()
avax_daily_ave



,Average Daily Price
Date,
2020-09-26,4.622540
2020-09-27,4.576908
2020-09-28,4.427049
2020-09-29,4.169829
2020-09-30,4.230119
...,...
2022-02-22,72.352580
2022-02-23,74.136868
2022-02-24,74.888994


In [76]:
def plot_close_prices():
    """
    Plot the closing prices of AVAX in plotly.
    """
    fig = px.line(avax_daily_ave, y='Average Daily Price', title='AVAX Average Daily Prices')
    fig.update_traces(line_color='#e84142')
    fig.show()
plot_close_prices()

In [77]:
def decomp():
    """
    Perform seasonal decomposition on the avax_daily_ave dataframe.
    Plot the observed, trend, and seasonal components in plotly express.
    """
    avax_decomp = sm.tsa.seasonal_decompose(avax_daily_ave['Average Daily Price'], model='additive')
    fig = px.line(avax_decomp.observed, title='AVAX Average Daily Prices')
    fig.update_traces(line_color='#e84142')
    fig.show()
    fig = px.line(avax_decomp.trend, title='AVAX Average Daily Prices Trend')
    fig.update_traces(line_color='#e84142')
    fig.show()
    fig = px.line(avax_decomp.seasonal, title='AVAX Average Daily Prices Seasonal')
    fig.update_traces(line_color='#e84142')
    fig.show()
    fig = px.line(avax_decomp.resid, title='Seasonal Residuals')
    fig.update_traces(line_color='#e84142')
    fig.show()

decomp()

# Train/Test Split

# Time-series Model


In [78]:
if 'time' not in avax_daily_ave.columns:
    avax_daily_ave.index.rename("time", inplace=True)
    avax_daily_ave.reset_index(inplace=True)



def build_baseline_model():
    """
    Baseline ARIMA model for avax_daily_ave using kats.models.sarima.
    """
    params = SARIMAParams(
    p = 2, 
    d=1, 
    q=1, 
    trend = 'ct', 
    seasonal_order=(1,0,1,12)
    )
    
    ts = kats.consts.TimeSeriesData(avax_daily_ave)
    model = kats.models.sarima.SARIMAModel(data = ts, params = params)
    model.fit()
    fcst = model.predict(steps=30)
    #model.summary()
    model.plot(style = ['r'])
    plt.show()
build_baseline_model()
avax_daily_ave

/home/austin/anaconda3/envs/avax_env/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals



TypeError: plot() got an unexpected keyword argument 'style'

# Forecasting Avax Price for the next 10 days

# Compare the predicted price with the actual price